# Colab training notebook

This notebook runs the existing training code from the repository on Google Colab (GPU runtime). It installs required packages, mounts Google Drive optionally for dataset/checkpoints, and calls the train entrypoint in `src/train.py` so functionality remains unchanged.

Notes:
- Make sure your dataset folder has `dr_labels.csv` and a `DR_images/` subfolder.
- You can either upload the `data/` folder to Colab session storage, or mount Google Drive and point `--data-dir` to a folder on Drive.
- If you prefer to run from a GitHub repo, upload this workspace to a public GitHub and use the git clone cell below.

In [16]:
# Install required packages (run once)
# Runtime: select 'Runtime' -> 'Change runtime type' -> Hardware accelerator: GPU
# Install CUDA-enabled PyTorch (Colab GPU runtime usually supports the latest stable CUDA).
# This installs torch and torchvision with GPU support, then the remaining requirements.
!pip install -q torch torchvision --extra-index-url https://download.pytorch.org/whl/cu117
!pip install -r requirements_colab.txt --quiet

In [12]:
%cd /content

/content


In [13]:
mkdir /content/Diabetic-Retinopathy

In [3]:

!rm -rf /content/Diabetic-Retinopathy  # remove any previous copy
!git clone https://github.com/Ojasvsakhi/Diabetic-Retinopathy.git /content/Diabetic-Retinopathy
%cd /content/Diabetic-Retinopathy

# Option 3: Mount Google Drive to access large datasets or persist checkpoints
from google.colab import drive
drive.mount('/content/drive')
import os
# Your dataset is in My Drive in the folder named exactly 'DR dataset'
DATA_DIR = '/content/drive/MyDrive/DR Dataset'
if not os.path.exists(DATA_DIR):
    print('Warning: expected DATA_DIR not found:', DATA_DIR)
else:
    print('Using DATA_DIR =', DATA_DIR)

Cloning into '/content/Diabetic-Retinopathy'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 37 (delta 11), reused 32 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 19.44 KiB | 19.44 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/Diabetic-Retinopathy
Mounted at /content/drive
Using DATA_DIR = /content/drive/MyDrive/DR Dataset


In [6]:


import sys, os
# make sure src is on PYTHONPATH (adjust if you cloned into a subfolder)
repo_src = os.path.join(os.getcwd(), 'src')
if os.path.exists(repo_src):
    sys.path.insert(0, repo_src)
else:
    sys.path.append('src')

# parse args similar to running `python -m src.train`
from argparse import Namespace
from src.train import train

args = Namespace(
    data_dir=DATA_DIR,  # using your Drive folder 'DR dataset'
    epochs=20,
    batch_size=16,
    img_size=224,
    lr=1e-4,
    num_workers=2
)

# Kick off training
train(args)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Validation: 100%|██████████| 7/7 [00:10<00:00,  1.44s/it]


Epoch 1 validation -- acc: 0.5354 macro-F1: 0.2865


Validation: 100%|██████████| 7/7 [00:08<00:00,  1.26s/it]


Epoch 2 validation -- acc: 0.5556 macro-F1: 0.3301


Validation: 100%|██████████| 7/7 [00:09<00:00,  1.43s/it]


...

Epoch 18 validation -- acc: 0.6465 macro-F1: 0.5529


Validation: 100%|██████████| 7/7 [00:10<00:00,  1.45s/it]


Epoch 19 validation -- acc: 0.7172 macro-F1: 0.6314


Validation: 100%|██████████| 7/7 [00:08<00:00,  1.25s/it]

Epoch 20 validation -- acc: 0.6263 macro-F1: 0.4901
